# This script is to identify words by the phonetic


## Install required packages
Run the next cell only if you don't have the packages already installed

In [ ]:
! pip install scipy matplotlib sounddevice wave playsound tqdm librosa pyphen opencv-python pandas parselmouth

To add sugestions and auto complete in jupyter notebook

In [ ]:
!pip install jupyter_contrib_nbextensions
!pip install jupyter_nbextensions_configurator
!jupyter contrib nbextension install --user 
!jupyter nbextensions_configurator enable --user
# after packages are installed
#  - go to home page
#  - click on nbextensions tab
#  - unckeck disable configuration for nbextensions without explicit compatibility
#  - put a check on Hinterland

## <center><span style='color :blue' >Create dataset</span>



In [ ]:
# import required packages
from os.path import join, exists
import sounddevice as mic
from scipy.io.wavfile import write
import os


def create_directories():
    # create folder that contain recorded words
    folder_name = 'Dataset'

    # create subfolders with the name of each group member
    sub_folders = ['Andre', 'Marcelo', 'Jorge']
    
    # create subsubfolders with the word
    sub_sub_folders = ['chata','chapa','chave','lata','lapa','lava','casa','capa','cave','chuta','chupa',
                       'chuva','farta','farpa','farda','ripa','rita','rica']
    
    principal_directory = join(os.getcwd(), folder_name)

    # check if the principal folder already exist
    if not exists(principal_directory):
        os.mkdir(principal_directory)
        
    # check if the sub folder already exist
    for sub_folder in sub_folders:
        if not exists(join(principal_directory, sub_folder)):
            os.mkdir(join(principal_directory, sub_folder))
            
        # check if the sub sub folder already exist
        for sub_sub_folder in sub_sub_folders:
            if not exists(join(principal_directory, sub_folder, sub_sub_folder)):
                os.mkdir(join(principal_directory, sub_folder, sub_sub_folder))
                

def record_audio():
    # get the word and the person name by receive input from keyboard
    person_name = input('Please enter your name:\n')
    audio_name = input('Please enter the word that you will say:\n')
    print( f'{person_name} will say the word {audio_name}')
    
    # directory to save the audio recorded
    save_name = join(os.getcwd(),'Dataset',person_name) 
    if not exists(save_name): # check if the folder exist
        print("""The possible names are:
        - Andre 
        - Marcelo 
        - Jorge """)
        record_audio() # try record audio again
    

    fs = 44100  # Sample rate of the audio file
    seconds = 3  # Duration of recording
    
    print('Please say the word')
    myrecording = mic.rec(int(seconds * fs), samplerate=fs, channels=2)
    mic.wait()  # Wait until recording is finished
    write(join(save_name , (audio_name + '.wav')), fs, myrecording)  # Save as WAV file 
    print(f'{audio_name} is saved in {person_name} folder')
# record_audio()  
create_directories()

## <center> <span style='color :blue' >Sound Analysis</span>

## Import all required libraries


In [ ]:
# Run this cell to import all the required libraries in code below
from librosa import load, get_duration, display, feature, magphase, frames_to_time, onset, autocorrelate, effects, stft, amplitude_to_db, times_like, pyin
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, accuracy_score, classification_report
from scipy.signal import lfilter, lfilter_zi, filtfilt, butter, freqz, spectrogram, get_window, find_peaks
import pyphen, math, time, glob, os, cv2, pickle, parselmouth
from sklearn.model_selection import train_test_split
from librosa.util import fix_length
from parselmouth.praat import call
# from pydub import AudioSegments
from os.path import join, exists
import matplotlib.pyplot as plt
import IPython.display as ipd
from sklearn.svm import SVC
import scipy.fftpack as fft
from math import log as ln
from tqdm import tqdm
from math import exp
import pandas as pd
import numpy as np

### Read and View the graph of each audio

In [ ]:
def get_audios():
    words_examples = [] # list initialization
    audio_files = glob.glob('Dataset/*')  # get all folders inside Dataset folder
    
    for file in audio_files:
        words = glob.glob(f'{file}/*') # get all examples inside each person folder    
        for word in words:
            examples = glob.glob(f'{word}/*') # get all examples inside each person folder    
            if len(examples): # check if exist any example in each person
                for example in examples:
                    if example.endswith('.wav'):
                        words_examples.append(example) # append all examples in list

    return words_examples 

def read_audio_signal(audio_file):    
    audio, fs = load(audio_file) # read audio file with fs = 22050 in mono mode

    padded_audio = fix_length(audio, size=4*fs)
    duration = get_duration(y=padded_audio, sr=fs) # get the duration of the audio in seconds
    
    duration_time = np.linspace(0, duration, audio.shape[0]) # create time vector to x-scale in graph

    number_points = int(fs * duration) # signal size in points
    
    return fs, padded_audio, duration_time, number_points
        
def show_orginal_audios(): # show original wave of each word
    start_time = time.time()
    audio_examples = get_audios() # call function to get all audios in dataset
    size =  round(np.sqrt(len(audio_examples)))
    for i in tqdm(range(len(audio_examples))):
        print(i)
        fs, audio_readed, duration_time, _ = read_audio_signal(audio_examples[i]) # read audio example
        
        # define ghapics parameters 
        plt.subplot(size, size, i+1)
        plt.rcParams['figure.figsize'] = [15, 7]
        plt.tight_layout(pad=3.0)

        # show graph with original audio signal
        plt.plot(duration_time,audio_readed)
        # plt.xlabel('Time [s]')
        # plt.ylabel('Amplitude')
        # plt.title(audio_examples[i].split('\\')[1] + ' saying ' +  audio_examples[i].split('\\')[-1].split('.')[0])
    plt.show()   
    current_time = time.time()
    elapsed_time = current_time - start_time
    print(elapsed_time)
    
# show_orginal_audios()   
# get_audios()

In [ ]:
audio, fs = load('Dataset/Andre/casa/0.wav') # read audio file with fs = 22050 in mono mode
y, sr = librosa.load('Dataset/Andre/casa/0.wav')
librosa.get_duration(y=y, sr=sr)
padded_audio = fix_length(audio, size=4*fs)
duration = librosa.get_duration(y=padded_audio,sr=fs) # get the duration of the audio in seconds
display.waveshow(audio, sr=fs)

display.waveshow(padded_audio, sr=fs, alpha = 0.5)


## Filter the original signal between typical voice frequencies

- The most common voice frequencies are between 60 Hz to 7000 Hz "https://www.atlasdasaude.pt/publico/content/voz";


- Due to voice has a low and high range will be aplied a band-pass filter;


- The filter will be applied forward and backward to avoid delays in the filtered signal when compared with the original.

In [ ]:
def filter_original_voice(filter_response = False, visualize_graph = False):
    start_time = time.time()

    filtered_audios = [] # initialize list
    
    audio_examples = get_audios() # call function to get all audios in dataset
    size =  round(np.sqrt(len(audio_examples)))
    for i in tqdm(range(len(audio_examples))):
        fs, audio_readed, duration_time, _ = read_audio_signal(audio_examples[i]) # read audio example
        
        nyq = 0.5 * fs # nyquist frequency
        
        # normalize cutoff frequency
        low = 60 / nyq 
        high = 7000 / nyq 
        
        b, a = butter(6, [low, high], btype='band') # define 5th order band-pass butterworth filter 

        w, h = freqz(b, a) # compute frequency response

        zi = lfilter_zi(b, a) # set initial state of the filter
        z, _ = lfilter(b, a, audio_readed, zi=zi*audio_readed[0]) # filter signal from left to right
        z2, _ = lfilter(b, a, z, zi=zi*z[0]) # filter signal from right to left
        audio_filtered = filtfilt(b, a, audio_readed) # filter signal forward and backward
        
        filtered_audios.append([audio_examples[i], audio_filtered, fs]) # save name and filtered audio in list
        size =  round(np.sqrt(len(filtered_audios))) # get the sqrt of the list size to use in graphics parameters
        if visualize_graph:
            
            # define graphics parameters 
            plt.subplot(size, size, i+1)
            plt.rcParams['figure.figsize'] = [15, 7]
            plt.tight_layout(pad=3.0)

            # show graph with original audio signal
            plt.plot(duration_time,audio_readed, 'r', linewidth = 3)

            # show graph with filtered audio signal
            plt.plot(duration_time,audio_filtered, 'g', alpha = 0.8)
            plt.legend(('original signal','Filtered signal'), loc = 3, prop={'size': 7})  
            plt.grid(True)
            plt.xlabel('Time [s]')
            plt.ylabel('Amplitude')
            plt.title(audio_examples[i].split('\\')[1] + ' saying ' +  audio_examples[i].split('\\')[-1].split('.')[0])
            
    if visualize_graph:
        plt.show()
    
    # show frequency response
    if filter_response:
        plt.plot((fs * 0.5 / np.pi) * w, 20 * np.log10(abs(h)), 'b')
        plt.ylabel('Amplitude [dB]', color='b')
        plt.xlabel('Frequency [rad/sample]')   
        plt.title('Frequency Response')  
        plt.grid(True)
        plt.xticks(range(0, 12000, 1000))
        plt.ylim(-500, 10)

        # Clearer view of frequency response at high cutoff frequency
        ax1 =  plt.axes([0.2, 0.3, 0.2, 0.2])
        ax1.plot((fs * 0.5 / np.pi) * w, 20 * np.log10(abs(h)), 'r')
        plt.grid(True)
        plt.title('Zoom in high cutoff')   
        plt.xlim(5000,8000)
        plt.ylim(-10, 10)
        
        # Clearer view of frequency response at low cutoff frequency
        ax2 =  plt.axes([0.6, 0.3, 0.2, 0.2])
        ax2.plot((fs * 0.5 / np.pi) * w, 20 * np.log10(abs(h)), 'r')
        plt.grid(True)
        plt.title('Zoom in low cutoff')   
        plt.xlim(0,100)
        plt.ylim(-200, 10)

        plt.show()
    current_time = time.time()
    elapsed_time = current_time - start_time
    print('All audios are filtered')
    return filtered_audios, size


# call function to visualize frequency response and graphic of each filtered audio
filtered_audios, _ = filter_original_voice(filter_response= False, visualize_graph=False) 

### View spectrogram to see where are the power and silence in filtered signal


In [ ]:
# import required packages

def see_spectrogram_scipy():
    
    filtered_audios = filter_original_voice(filter_response = False, visualize_graph = False) # call function to get all filtered audios
    for i in range(len(filtered_audios)):
        
        name = filtered_audios[i][0] # get the word and the person
        wave = filtered_audios[i][1] # get audio wave
        fs = filtered_audios[i][2] # get sample rate
        
        # define ghapics parameters 
        plt.subplot(4, 4, i+1)
        plt.rcParams['figure.figsize'] = [15, 15]
        plt.tight_layout(pad=3.0)

        # perfom spectogram analysis
        f, t, S1 = spectrogram(wave,fs,window='flattop', nperseg=fs//10, noverlap=fs//20, scaling='density', mode='magnitude')
        
        # show graph with spectogram of each audio
        plt.pcolormesh(t, f[:800], S1[:800][:])
        plt.xlabel('time(s)')
        plt.ylabel('frequency(Hz)')
        plt.title(name.split('\\')[1]+ ' saying ' + name.split('\\')[-1].split('.')[0])
        
    plt.show()
     
        
def see_spectrogram_librosa (y_scale = 'log'):
    filtered_audios, size = filter_original_voice(filter_response = False, visualize_graph = False) # call function to get all filtered audios
    for i in tqdm(range(len(filtered_audios))):
        
        name = filtered_audios[i][0] # get the word and the person
        wave = filtered_audios[i][1] # get audio wave
        fs = filtered_audios[i][2] # get sample rate
        
        
        X = stft(wave) # aply stft to filtered signal
        Xdb = amplitude_to_db(abs(X)) # convert signal only to positive values and convert amplitude to db
        
        # define ghap parameters 
        plt.subplot(size, size, i+1)
        plt.rcParams['figure.figsize'] = [15, 7]
        plt.tight_layout(pad=3.0)
        
        # show graph with spectogram of each audio
        librosa.display.specshow(Xdb[:800], sr=fs, x_axis='time', y_axis= y_scale)
        
    plt.colorbar()
    
see_spectrogram_librosa()

### Split the audio in silence and non-silence

In [ ]:
def find_non_consecutive(lst, distance = 3):
    
    # for loop to iterate the list   
    for i,j in enumerate(lst,lst[0]): 
        
        # get the non consecutive values which represent words in the audio
        if i!=j and j - i > distance: 
            return [i,j], j
                

def calculate_rms(show_graph = False):  
    
    get_exp_value = False # bool variable to indicate if exponential value is already known
    audio_flag= []
    frame_length = 2048
    hop_length = 1024
    
    for i in tqdm(range(len(filtered_audios))):
        
        name = filtered_audios[i][0] # get the word and the person
        wave = filtered_audios[i][1] # get audio wave
        fs = filtered_audios[i][2] # get sample rate    
        
#         print(name, i)
        # determine average power in each frame
        rms = feature.rms(y=abs(wave), frame_length=frame_length, hop_length=hop_length, center=True)[0]
        
        rms_rounded = np.round(rms, decimals=2) # round rms values in 2 decimals
        
        if not get_exp_value:
            difference_value = (len(wave)/len(rms_rounded))
            x = math.floor(math.log10(difference_value))  # x = 3
            exp_value = 1 * 10**x                       
            get_exp_value = True
            
        # list to return values where audio is
        silence_time = []
        for rms_index in range(len(rms_rounded)):
                if rms_rounded[rms_index] == 0:
                    silence_time.append(rms_index)

        marker_phrase, index = find_non_consecutive(silence_time) # get the limits of phrase in audio segment    
        marker_words_limit = silence_time.index(index) # find the index of the limit in marker phrase
        
        del silence_time[:marker_words_limit] # delete all values in list until the limit to research
        revaluate_word_position, _ = find_non_consecutive(silence_time) # get the limits of word in audio segment

        phrase_begin = np.multiply(np.min(marker_phrase), exp_value) # get the value where phrase begin
        phrase_end = np.multiply(np.max(marker_phrase), exp_value) # get the value where phrase end
        
        word_begin = np.multiply(np.min(revaluate_word_position), exp_value) # get the value where word begin     
        word_end = np.multiply(np.max(revaluate_word_position),exp_value) # get the value where word end

        audio_flag.append([phrase_begin,phrase_end,word_begin,word_end])    
        
        if show_graph:
            # get number of points in signal and the respective time
            frames_f = range(len(rms_rounded))
            time_scale = frames_to_time(frames_f, sr=fs, hop_length=hop_length)

            # define ghap parameters 
            plt.subplot(size, size, i+1)
            plt.rcParams['figure.figsize'] = [15, 15]
            plt.tight_layout(pad=3.0)

            # show graph with rms of each audio
            plt.plot(time_scale, rms_rounded, 'g')
            plt.title(name.split('\\')[1]+ ' saying ' + name.split('\\')[-1].split('.')[0])
    
    if show_graph:
        plt.show()
    
    print('End of rms calculation')
    return audio_flag

audio_flag = calculate_rms()


## Extract number of syllables in audio

- To count the number of syllables in the word only the audio segment that contains the word will be used;


- To implemenent this counter will be used *__"OnSet detection"__* from librosa;


- The  *__"OnSet detection"__* method is used to find the moment where musical notes begin. As our voice is composed of different frequencies, and those frequencies can be atributed to musical notes, this method will help to find when each syllable begin; "https://repositorio.iscte-iul.pt/handle/10071/5991"




In [ ]:
def split_audio_syllables():
        
    splited_phonemes = []
    for i in tqdm(range(len(filtered_audios))): 

        name = filtered_audios[i][0] # get the word and the person
        wave = filtered_audios[i][1] # get audio wave
        fs = filtered_audios[i][2] # get sample rate 
        y = wave[audio_flag[i][2]:audio_flag[i][3]] # get only the

        odf = onset.onset_strength(y=y, sr=fs, hop_length=512)
        ac = autocorrelate(odf/odf.max(), max_size= fs // 512)
        peaks, _ = find_peaks(ac, height=0.15)
    #     print(len(ac),  name, i)

        if len(peaks):
            syllable_limit =  audio_flag[i][2] + math.floor(peaks[-1]/2) * 1000 + 1000
        else:
            syllable_limit =  audio_flag[i][2] + math.floor((len(y)/1000)/2) * 1000 + 1000
        
        splited_phonemes.append([audio_flag[i][2], syllable_limit, audio_flag[i][3], name])

    print('All words are splited in phonemes')
    
    return splited_phonemes

splited_phonemes = split_audio_syllables()

### Put the values of splited audios in csv file
This is only to facilitate the interpretation


In [ ]:
from librosa import frames_to_time
from librosa import feature
import pandas as pd

frame_length = 2048
hop_length = 1024

part1 = []
part2 = []
part3 = []
part4 = []
names = []

for i in tqdm(range(len(filtered_audios))):
    
    name = filtered_audios[i][0] # get the word and the person
    wave = filtered_audios[i][1] # get audio wave
    fs = filtered_audios[i][2] # get sample rate    
    
    # determine average power in each frame
    rms = feature.rms(abs(wave), frame_length=frame_length, hop_length=hop_length, center=True)[0]

    rms_rounded = np.round(rms, decimals=2) # round rms values in 2 decimals
    
    difference_value = (len(wave)/len(rms_rounded))
    x = math.floor(math.log10(difference_value))  # x = 3
    exp_value = 1 * 10**x    
    
    silence_time = []
    for rms_index in range(len(rms_rounded)):
            if rms_rounded[rms_index] == 0:
                silence_time.append(rms_index)
            
    marker_phrase, index = find_non_consecutive(silence_time) # get the limits of phrase in audio segment    
    marker_words_limit = silence_time.index(index) # find the index of the limit of marker phrase
    del silence_time[:marker_words_limit] # delete all values in list until the limit to research
    revaluate_word_position, _ = find_non_consecutive(silence_time) # get the limits of word in audio segment

    phrase_begin = np.multiply(np.min(marker_phrase), exp_value) # get the value where phrase begin
    phrase_end = np.multiply(np.max(marker_phrase), exp_value) # get the value where phrase end

    word_begin = np.multiply(np.min(revaluate_word_position), exp_value) # get the value where word begin     
    word_end = np.multiply(np.max(revaluate_word_position),exp_value) # get the value where word end

    
    names.append(name)
    part1.append(phrase_begin)
    part2.append(phrase_end)
    part3.append(word_begin)
    part4.append(word_end)
    
df = pd.DataFrame(np.column_stack([names, part1, part2,part3,part4]), 
                               columns=['voiceID', 'limit1', 'limit2', 'limit3', 'limit4']) 
# Write out the updated dataframe
df.to_csv("processed_results.csv", index=False)    


#### Normalize audio values after pre-processement


In [ ]:
def normalize_audio(audio):
    positive_audio = abs(audio) # convert audio to absolute values
    audio = positive_audio / np.max(positive_audio) # divide audio values by max value in audio to normalize
    
    return audio

#### Aply framimg in audio
- This process will help to avoid distortion in the FFT. 


Due to the audio being a non-stationary signal this will produce FFT distortions, so if we consider that audio is a stationary signal for short periods we will be capable to avoid those distortions. To implement this assumption, the signal will be divided into short frames. However, we need those frames to overlap to don't lose information on the edges of each frame.


In [ ]:
def framing_audio(audio, FFT_size=2048, hop_size=10, sample_rate=22050):
  
    audio = np.pad(audio, int(FFT_size / 2), mode='reflect') # padding the signal to ensure that all samples in frame have equal lenght 
    frame_len = np.round(sample_rate * hop_size / 1000).astype(int) # convert frame lenght from seconds to samples
    frame_num = int((len(audio) - FFT_size) / frame_len) + 1 # convert frame number from seconds to samples
    frames = np.zeros((frame_num,FFT_size)) # define structure of frames
    
    for n in range(frame_num):
        frames[n] = audio[n*frame_len:n*frame_len+FFT_size]
    
    return frames

def fft_spectrum(audio_framed, hop_size=544 , FFT_size=2048):    

    ''' create window to limit audio frammed
        hann window is used because this method is useful
        to smoothing discontinuities in the edges of each frame'''
    
    window = get_window("hann", FFT_size, fftbins=True) 

    audio_windowed = audio_framed * window # aply the smooth operation in audio framed

    audio_windowedT = np.transpose(audio_windowed) # aply the tranpose to performed the fft
    audio_fft = np.empty((int(1 + FFT_size // 2), audio_windowedT.shape[1]), dtype=np.complex64, order='F') # define structure of fft
    for n in range(audio_fft.shape[1]):
        audio_fft[:, n] = fft.fft(audio_windowedT[:, n], axis=0)[:audio_fft.shape[0]]

    audio_fft = np.transpose(audio_fft) # invert the shape of resultant fft to the same of audio_windowed

    audio_power = np.square(np.abs(audio_fft)) # get the power of signal
    
    return audio_power

## MFCCs extraction

To calculate the mfccs of each syllable we will follow the next steps:

    - Extract the maximum and minimum frequencies in the signal based on spectral band width;
    
    - Create filter banks based on these frequencies;
    
    - Multiply filters with signal in power;
    
    - Calculate the mfccs.

#### Get the maximum and minimum frequencies of spectrum

In [ ]:
def maximum_minimum_hz(audio, show_graph = False):
        
    S, _ = magphase(stft(y=audio)) # perform stft in the signal
    spec_bw = feature.spectral_bandwidth(S=S) # obtain the spectral band to get max and min frequencies
    
    if show_graph:
        plt.figure()
        plt.subplot(2, 1, 1)
        plt.semilogy(spec_bw.T, label='Spectral bandwidth')
        plt.ylabel('Hz')
        plt.xticks([])
        plt.xlim([0, spec_bw.shape[-1]])
        plt.legend()
        plt.subplot(2, 1, 2)
        display.specshow(amplitude_to_db(S, ref=np.max),
                                 y_axis='log', x_axis='time')
        plt.title('log Power spectrogram')
        plt.tight_layout()
        plt.show()
        
    return np.round(np.min(spec_bw), decimals=4), np.round(np.max(spec_bw), decimals=4)

#### Construction of filter bank

In [ ]:
# convert hz to mel scale
def freq_to_mel(freq):
    return np.round(1125 * ln(1.0 + freq / 700), decimals=4)

# convert mel scale to hz
def mel_to_freq(mels):
    return 700 * (exp(mels / 1125) - 1)


def get_filter_points(fmin, fmax, mel_filter_num, FFT_size, sample_rate=22050):
    freqs = []
    filter_bank = []
    
    fmin_mel = freq_to_mel(fmin) # convert initial frequency limit from Hz to mel
    fmax_mel = freq_to_mel(fmax) # convert final frequency limit from Hz to mel
    
    print(f'Minimum mel frequency: {fmin_mel}')
    print(f'Maximum mel frequency: {fmax_mel}')
     
    mels = np.linspace(fmin_mel, fmax_mel, num=mel_filter_num + 2) # construct linear array between the frequencies limits
    
    [freqs.append(mel_to_freq(mel)) for mel in mels]  # convert the linear array from mel scale to Hz    
    [filter_bank.append(np.floor((FFT_size + 1) / sample_rate * freq).astype(int)) for freq in freqs]  # normalize values to FFT size

    filters = np.zeros((len(filter_bank)-2,int(FFT_size/2+1))) # define the structure of filters

    for n in range(len(filter_bank)-2):
        
        #define the lower limit of the filter
        filters[n, filter_bank[n] : filter_bank[n + 1]] = np.linspace(0, 1, filter_bank[n + 1] - filter_bank[n]) # this function will produce a come up from 0 to 1 in the begining point until the after
        
        #define the upper limit of the filter
        filters[n, filter_bank[n + 1] : filter_bank[n + 2]] = np.linspace(1, 0, filter_bank[n + 2] - filter_bank[n + 1]) # this function will produce a come down from 1 to 0 in the final point until the afte
        
    
    # saw in the librosa library
    freqs = np.array(freqs) # convert list to array
    
    enorm = 2.0 / (freqs[2:mel_filter_num+2] - freqs[:mel_filter_num]) # divide the triangular MEL filter by the width of the MEL band (area normalization)
    filters *= enorm[:, np.newaxis]
    
    return filters

#### Calculation of mfccs

In [ ]:
def mfccs(filters, audio_power, syllable):
        
    audio_filtered_mfccs = np.dot(filters, np.transpose(audio_power)) # aply the filters into the spectrum calculated above
    audio_log = 10.0 * np.log10(audio_filtered_mfccs) # convert to log scale
    audio_log.shape


    mel_coeficients = 40

    coefficients = np.transpose(fft.dct(audio_log, type=3)) # will be calculated the type 3 dct to distinguish from high to low frequencies
    mfccs_values = np.dot(coefficients, audio_log) # aply the number of coeficients in the spectrum

    plt.figure(figsize=(15,5))
    plt.imshow(mfccs_values, aspect='auto', origin='lower',cmap='coolwarm')
    plt.savefig(syllable)
    plt.close()  

## After we split the word in phonemes let's create a new phonetic dataset !!!

In [ ]:
def split_written_word(word):
    dic = pyphen.Pyphen(lang='pt')
    return dic.inserted(word).split('-')

def create_phonetic_dataset():
    hop_size = 10 # ms
    FFT_size = 2048 # size of frame
    pos = 0
    name_example_0 = 0
    name_example_1 = 0

    principal_folder = 'Dataset'
    sub_folder = 'Phonemes'
    if not exists(join(principal_folder,sub_folder)): # check if the folder already exists else create 
        os.mkdir(join(principal_folder, sub_folder))

    for i in tqdm(range(len(splited_phonemes))):
        wave = filtered_audios[i][1] # get audio wave
        name = filtered_audios[i][0] # get the word and the person
        name = name.split('\\')[2] # get the word said
        phonetics = split_written_word(name) # split the word in syllables

        while pos != len(splited_phonemes[i]) - 2:
            audio = wave[splited_phonemes[i][pos]:splited_phonemes[i][pos+1]] # get the wave per syllable
            if pos == 0:
                number = name_example_0
                phonetic = phonetics[0] # get the corresponding syllable from wave
                name_example_0 += 1
            else:
                number = name_example_1
                phonetic = phonetics[1] # get the corresponding syllable from wave
                name_example_1 += 1

            if not exists(join(principal_folder,sub_folder, phonetic)): # check if the folder already exists else create 
                os.mkdir(join(principal_folder, sub_folder,phonetic))  

            audio_normalized = normalize_audio(audio) # normalize audio
            audio_framed = framing_audio(audio_normalized, FFT_size=FFT_size, hop_size=hop_size, sample_rate=22050) # framming the audio
            audio_power = fft_spectrum(audio_framed, hop_size=hop_size, FFT_size=FFT_size) # perform fft and convert to power

            freq_min, freq_max = maximum_minimum_hz(audio, show_graph = False) # get the minimun and the maximum frequencies of the audio
            filters_to_mfcc =  get_filter_points(freq_min, freq_max, mel_filter_num=10, FFT_size=FFT_size, sample_rate=22050) # construct filter bank

            mfccs(filters_to_mfcc, audio_power, join(principal_folder, sub_folder,phonetic,str(number)))  # calculate the mfccs      
            pos += 1
        pos = 0
        
create_phonetic_dataset()    

## Train SVM model

In [ ]:
def train_model(dataset):
    
    X = [] # list initialization
    y = [] # list initialization

    for samples, label in dataset:
        X.append(samples)
        y.append(label)
    
    X= np.array(X).reshape(len(dataset),-1) # reshape the samples

    X = X/255.0 # normalize samples between 0 and 1
    y = np.array(y)

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, stratify=y) # split data to train and test

    print(f'The model will be trained with {X_train.shape[0]} and tested with {X_test.shape[0]} samples.','\n')
    
    start = time.time()   
    svc = SVC(kernel='linear',gamma='auto') # define the model
    svc.fit(X_train, y_train) # train the model
    end = time.time()
     
    print(f'The model took {(end-start)/60} minutes to train.', '\n')    
          
    filename = 'final1_model.sav'
    pickle.dump(svc, open(filename, 'wb')) # save model
    
    evalute_model(X_test, y_test, filename)
    
def evalute_model(X_test, y_test, filename):
    
    model = pickle.load(open(filename, 'rb')) # load model
    predictions = model.predict(X_test) # testing test set
    
    report_dict = classification_report(y_test, predictions, output_dict=True) # get the classification report
    report = pd.DataFrame(report_dict)
    report.to_csv('reports/report.csv', index = False) # save the classification report
    
    print(f'Model has an accuracy of {accuracy_score(y_test,predictions)* 100} % on test set')
    
    cm = confusion_matrix(y_test, predictions, labels=model.classes_) # create confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_) # convert confusion matrix to graph
    disp.plot()
    plt.savefig('Confusion matrix')
    
def pre_process_data(folder):
    training_data = []   
    for file in dataset_folder:
        images = glob.glob(f'{file}/*') # get all examples inside each phoneme folder 
        for image in images:
            example = cv2.imread(image, cv2.IMREAD_GRAYSCALE) # read img and convert to gray scale
            example = cv2.resize(example,[360,480]) # resize img
            training_data.append([example,file.split('\\')[1]])
    
    print('Dataset is pre processed.')      
    print(f'Dataset contains a total of {len(training_data)} examples.', '\n')
    train_model(training_data)
    
dataset_folder = glob.glob('Dataset/Phonemes/*')  # get all folders inside Dataset folder
pre_process_data(dataset_folder)    

### Calculate the fundamental frequency of each person

- The pitch is the pulse frequency that a person makes to create sound like words or simply noise. So if we know this frequency we will be capable to identify the person who is speaking;


- Considering all people (children, men, and women) the typical range of  __f0 is between 80 Hz and 500 Hz;__


- In the cells below will be shown how it's possible to identify the pitch (f0) of each person. 


- Pitch will be calculated with three methods:
    1. __PYin__ method;
    2. __Cepstrum__ method.
    3. __Parselmouth library__ (praat method).
    
<img src="resources/voice-authentication-01.png" width="460" height="460">

### PYin method
This method derives from *__yin__* method, and both are available in librosa library. https://librosa.org/doc/main/generated/librosa.yin.html?highlight=yin#librosa.yin
https://librosa.org/doc/main/generated/librosa.pyin.html?highlight=pyin#librosa.pyin

Both methods are based on autocorrelation, however the way that those methods define the F0 is different. While yin use a single threshold during all signal, pyin use many thresholds based on probability and that's why it's able to predict if there or is not voice in signal.

<img src="resources/f0_estimation_ex.png" width="460" height="460">

In the image above it is possible to compare the two methods and verify if the yin predicts F0 where the signal has no voice, which is wrong and makes it impossible to obtain the most correct value of the person's F0. For this reason, we will __only use pyin.__

In [ ]:
def p_yin(y):
    
    frame_length = 512 # define frame lenght
    
    f0, voiced_flag, voiced_probs = pyin(y=y, fmin=80,fmax=500,frame_length=frame_length) # extract pith using pyin
    f0 = np.mean(f0[np.where(voiced_flag == True)])
    
    y_harmonic, y_percussive = effects.hpss(y) # extract harmonics and percurssives
    y_harmonic = np.round(np.mean(y_harmonic), decimals=5)
    y_percussive = np.round(np.mean(y_percussive), decimals=6)
    
    return f0, y_harmonic, y_percussive

### Cepstrum method
This method is the result of the inverse FFT, which means that instead of using frequency and spectrum will be used quefrency and cepstrum. The main advantage of this method is that the peak of fundamental frequency will be more highlighted when compared to the spectrum. The resultant graph shows two highlighted peaks, where the first marks the frequency and the second is the F0 of a person.

<img src="resources/cpe_vs_spe.png" width="460" height="460">

In [ ]:
def extract_cepstrum(audio, sample_freq):
    
    windowed_signal = np.hamming(audio.size) * signal # create windowed signal
    
    freq_vector = np.fft.fftfreq(frame_size, d=1/sample_freq) # create frequency vector to fft
    X = np.fft.fft(windowed_signal) # aply fft
    log_X = np.log(np.abs(X)) # convert fft to power spectrum
    
    cepstrum = np.fft.fft(log_X) # convert spectrum to cepstrum
    df = freq_vector[1] - freq_vector[0] # create a vector size
    quefrency_vector = np.fft.fftfreq(log_X.size, df) # get the quefrency values
    
    return quefrency_vector, cepstrum

def extract_ceps_f0(signal, sample_freq=22050, fmin=80, fmax=500):
    """Returns f0 based on cepstral processing."""
    quefrency_vector, cepstrum = extract_cepstrum(signal, sample_freq) # extract cepstrum
    
    valid = (quefrency_vector > 1/fmax) & (quefrency_vector <= 1/fmin) # get the peaks only in F0 range
    max_quefrency_index = np.argmax(np.abs(cepstrum)[valid]) # get the index of max peak
    f0 = 1/quefrency_vector[valid][max_quefrency_index] # ensure that the peak is in f0 range 
    
    return f0

### Parselmouth library
This code is taken from https://github.com/drfeinberg/PraatScripts.git

In [ ]:
def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # read the sound
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    
    return meanF0, stdevF0, hnr

### Extract fundamental frequency using the three methods

In [ ]:
def get_pitch():
    
    # lists initialization
    
    file_list = []
    mean_F0_praat = []
    sd_F0 = []
    hnr_list = []
    
    mean_F0_pyin = []
    mean_harms = []
    mean_percs = []
    
    mean_F0_ceps = []

    # Go through all the wave files in the folder and measure pitch
    for i in tqdm(range(len(filtered_audios))):
        wave = filtered_audios[i][1] # get the wave
        name = filtered_audios[i][0].split('\\')[1] # get the person name
        file_list.append(name) # make an ID list
        
        y = wave[audio_flag[i][0]:audio_flag[i][1]] # get only the marker words in audio

        # Parselmouth extraction
        sound = parselmouth.Sound(y)
        meanF0, stdevF0, hnr = measurePitch(sound, 80, 500, "Hertz")         
        mean_F0_praat.append(meanF0) 
        sd_F0.append(stdevF0)
        hnr_list.append(hnr)

        # PYin extraction
        f0, harmonic, percurssive = p_yin(y)
        mean_F0_pyin.append(f0)
        mean_harms.append(harmonic)
        mean_percs.append(percurssive)
        
        # Cepstrum extraction
        f0 = extract_ceps_f0(y)
        mean_F0_ceps.append(f0)
        
        
    praat = pd.DataFrame(np.column_stack([file_list, mean_F0_list, sd_F0_list, hnr_list]), 
                               columns=['voiceID', 'meanF0 _Hz', 'stdevF0 _Hz', 'HNR'])  #add these lists to pandas in the right order
    # Write out the updated dataframe
    praat.to_csv("pitch_range_praat.csv", index=False)    

    pyin = pd.DataFrame(np.column_stack([file_list, mean_F0_pyin, mean_harms, mean_percs]), 
                                   columns=['voiceID', 'meanF0_Hz', 'Harmonics', 'Percurssives'])  #add these lists to pandas in the right order
    # Write out the updated dataframe
    pyin.to_csv("pitch_range_praat.csv", index=False)    

    cepst = pd.DataFrame(np.column_stack([file_list, mean_F0_ceps]), 
                                   columns=['voiceID', 'meanF0_Hz'])  #add these lists to pandas in the right order
    # Write out the updated dataframe
    cepst.to_csv("pitch_range_praat.csv", index=False)    
    
get_pitch()    

In [ ]:
import IPython.display as ipd
i = 1
wave = filtered_audios[i][1] # get audio wave
print(splited_phonemes[i])
y = wave[44000:49000]
ipd.Audio(y,rate=22050)